In [1]:
import pandas as pd
import numpy as np

from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline


from lightgbm import LGBMClassifier

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import RepeatedStratifiedKFold


from skopt import BayesSearchCV
import seaborn as sns

%matplotlib inline

import keras
from keras.models import Sequential
from keras.layers import Dense

pd.set_option('display.max_columns', 500)

2021-09-30 21:58:43.563104: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-30 21:58:43.563123: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data = pd.read_csv('archive/train.csv')
test = pd.read_csv('archive/test.csv')


In [3]:
data.head()

,id,var1,var2,var3,var4,var5,var6,var7,var8,var9,var10,var11,var12,var13,var14,var15,var16,var17,var18,var19,var20,var21,var22,var23,var24,var25,var26,var27,var28,var29,var30,var31,var32,var33,var34,var35,var36,var37,var38,var39,var40,var41,var42,var43,var44,var45,var46,var47,var48,var49,var50,var51,var52,var53,var54,var55,var56,var57,var58,var59,var60,var61,var62,var63,var64,var65,var66,var67,var68,y
0,1,18,19,2853,29442,1386,2435,35,-999,3,63,6498,1166,2007,26,13,11,11,4,1547,26,2068,1,3,0,4,7,0,24,4,2,0,16,3,44,463,27,2,0,4,9,3,25,6,1,4,3,1,0,0,0,0,42,1,1,0.212414,0.137,0.833333,0.037822,0.058070,0.311441,0.142303,0.056146,0.632694,0.024054,0.253356,0.00603,0.132353,0.139706,1
1,8,4,110,1986,13684,7189,-999,-999,17,3,63,13989,497,2289,16,1,3,3,3,1797,16,2417,5,1,2,1,6,1,4,2,1,0,14,0,1,532,2,1,0,4,7,3,30,3,0,0,0,0,0,0,0,0,20,1,1,0.228784,0.308,0.305376,0.069325,0.248909,-999.000000,-999.000000,0.070991,0.773966,0.019315,-999.000000,-999.00000,0.147059,0.106618,0
2,30,0,39,1019,10232,678,791,16,-999,3,63,9739,562,641,10,34,34,10,4,511,10,664,5,3,2,1,7,0,3,4,1,0,11,1,3,81,26,3,0,4,3,3,23,5,1,0,0,0,0,0,0,0,12,1,1,0.204636,0.213,0.451613,0.018639,0.214520,-999.000000,0.200814,0.051046,0.980827,0.018536,-999.000000,-999.00000,0.382353,0.242647,0
3,43,20,39,1751,2689,8235,1042,13,10,1,14,2890,6541,811,8,59,60,23,5,624,8,839,0,0,0,0,1,1,19,1,2,0,8,0,11,414,27,0,0,4,10,3,22,10,1,1,1,0,1,1,0,0,36,1,1,0.208299,0.716,0.101075,0.204753,0.349421,-999.000000,0.352379,0.044301,0.951564,0.023684,0.363370,0.00201,0.147059,0.132353,0
4,46,7,44,2262,29428,6031,304,16,-999,3,63,13541,7238,260,10,55,56,20,5,1413,10,270,2,3,2,1,8,1,3,4,1,0,17,4,3,567,8,3,2,4,5,3,26,5,1,0,0,0,0,1,0,0,19,1,1,0.222896,0.596,0.101075,0.140394,0.189641,0.021226,0.226161,0.059125,0.906155,0.020733,-999.000000,-999.00000,0.455882,0.132353,1


In [10]:
desc = data.describe().T

f, ax = plt.subplots(figsize=(12, 30))

sns.heatmap(desc, annot=True, cmap='CMRmap_r', fmt='.02f',
            ax=ax, linewidths=5, cbar=False,
            annot_kws={'size': 16})

plt.xticks(size = 18)
plt.yticks(size = 12, rotation = 0)
plt.ylabel("Variables")
plt.title("Descriptive Statistics", size = 16)
plt.show()

In [3]:
drop = ['var9', 'var22', 'var23', 'var24', 'var25', 'var27', 'var29', 'var30', 
        'var31', 'var33', 'var39', 'var41', 'var44', 'var45', 'var46',
        'var47', 'var48', 'var49', 'var50', 'var51', 'var53', 'var54', 'y']

In [4]:
df = data[drop]

In [13]:
for col in df.columns[1:]:
  print(f"Col {col} = {df[col].unique()}")

Col var22 = [1 5 0 2 4 3]
Col var23 = [3 1 0 2 4]
Col var24 = [0 2 1]
Col var25 = [4 1 0 3 2]
Col var27 = [0 1]
Col var29 = [4 2 1 3 5 0]
Col var30 = [2 1 0]
Col var31 = [0 1]
Col var33 = [3 0 1 4 5 6 2]
Col var39 = [4 1 2 5 0 3]
Col var41 = [3 5 1 8 4 6 7 9 2 0]
Col var44 = [1 0]
Col var45 = [ 4  0  1  3  2  9  5  8  6 12  7 11 10 15 14 13]
Col var46 = [ 3  0  1  2  7  5  4  6  9  8 11 12 15 10 14]
Col var47 = [1 0 2 4 3 5]
Col var48 = [0 1 2 5 3 4 8 7 6 9]
Col var49 = [0 1]
Col var50 = [0 1]
Col var51 = [0 1]
Col var53 = [1 3 2 0]
Col var54 = [1 2 0 3]
Col y = [1 0]


# Create Model

## Spliting data and resampled

In [18]:
X = data.drop('y', axis=1)
y = data['y']

In [20]:
X_resampled, y_resampled = SMOTE().fit_resample(X, y)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.4, random_state=42)

In [22]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [23]:
X_train.shape

(13527, 69)

In [25]:
model = Sequential()
model.add(Dense(units=60, activation='relu', input_dim=69))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = 10, epochs = 100)

pred = model.predict(X_test)

1353/1353 [==============================] - 1s 991us/step - loss: 0.0585 - accuracy: 0.9793
Epoch 37/100
1353/1353 [==============================] - 1s 903us/step - loss: 0.0589 - accuracy: 0.9770
Epoch 38/100
1353/1353 [==============================] - 1s 906us/step - loss: 0.0526 - accuracy: 0.9809
Epoch 39/100
1353/1353 [==============================] - 1s 888us/step - loss: 0.0509 - accuracy: 0.9816
Epoch 40/100
1353/1353 [==============================] - 2s 1ms/step - loss: 0.0484 - accuracy: 0.9823
Epoch 41/100
1353/1353 [==============================] - 1s 944us/step - loss: 0.0476 - accuracy: 0.9823
Epoch 42/100
1353/1353 [==============================] - 1s 950us/step - loss: 0.0516 - accuracy: 0.9818
Epoch 43/100
1353/1353 [==============================] - 1s 947us/step - loss: 0.0475 - accuracy: 0.9831
Epoch 44/100
1353/1353 [==============================] - 1s 944us/step - loss: 0.0405 - accuracy: 0.9859
Epoch 45/100
1353/1353 [==============================] - 1s 

In [26]:
pred

array([[9.9995834e-01],
       [1.0000000e+00],
       [1.0000000e+00],
       ...,
       [2.2545039e-06],
       [1.9642413e-03],
       [9.8140013e-01]], dtype=float32)

In [28]:
pred = (pred > 0.5)

In [29]:
pred

array([[ True],
       [ True],
       [ True],
       ...,
       [False],
       [False],
       [ True]])

In [ ]:
# pred = model.predict(X_test)

## Ligth GBM

In [7]:
pipe = Pipeline([('model', LGBMClassifier())])

In [8]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('model', LGBMClassifier())])

In [9]:
pipe.score(X_test, y_test)

0.9111874930701852

## Cros Val Score

In [10]:
def cross_val(pipe, X, y, cv):
    return cross_val_score(pipe, X, y, scoring='f1', cv=cv, n_jobs=-1)

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

In [42]:
cross_val(pipe, X, y, cv).mean()

0.6412376047573998

## Tune Model, com bayesian optimize

In [11]:
params = {'model__learning_rate': [0.01], 'model__n_estimators': [8, 24, 50, 100, 500],
                   'model__num_leaves': [6, 8, 12, 16], 'model__boosting_type': ['gbdt'], 
                   'model__seed': [500],
                   'model__colsample_bytree': [0.65, 0.75, 0.8], 
                   'model__subsample': [0.7, 0.75], 'model__reg_alpha': [1, 2, 6],
                   'model__reg_lambda': [1, 2, 6]}

In [12]:
model_tune = BayesSearchCV(pipe, [(params, 20)], n_iter=-1, cv=cv)

NameError: name 'cv' is not defined

In [45]:
model_tune.fit(X_train, y_train)

In [1]:
model_tune.score(X_test, y_test)

NameError: name 'model_tune' is not defined

In [ ]:
cross_val(model_tune, X, y)

# Submission

In [129]:
y_pred = clf.predict(X_test)

In [130]:
f1_score(y_test, y_pred)

0.7388869026337742

In [131]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.83      0.77      4467
           1       0.81      0.68      0.74      4552

    accuracy                           0.76      9019
   macro avg       0.76      0.76      0.76      9019
weighted avg       0.76      0.76      0.76      9019



## Submission

In [15]:
pred = model.predict(X_test)

In [16]:
X_test.shape

(9019, 44)

In [17]:
test.shape

(21183, 69)

In [30]:
sub_pred = model.predict(test)

In [32]:
sub_pred

array([[1.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [31]:
sub_pred.shape

(21183, 1)

In [33]:
df = pd.DataFrame()

In [35]:
df = pd.DataFrame(sub_pred, columns=['predicted'])

In [36]:
df = pd.concat([test['id'], df], axis=1)

In [137]:
df

,id,predicted
0,0,1
1,2,1
2,4,0
3,7,1
4,15,1
...,...,...
21178,35297,0
21179,35298,0
21180,35300,1
21181,35302,0


In [44]:
df['predicted'] = df['predicted'].astype(bool)

In [45]:
df.to_csv('submission/14.csv', index=False)